In [1]:
!pip install opencv-contrib-python
!pip install --upgrade google-cloud-vision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 2.3 MB/s eta 0:00:00


In [2]:
import os
import io
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
import matplotlib.pyplot as plt

import cv2
from google.cloud import vision

In [3]:
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)

    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)

            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])

        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [4]:
# 이미지에 텍스트 표시를 위한 함수 정의
def putText(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_converted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_converted)

    # 기본 폰트를 로드
    image_font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)

    draw.text((x, y), text, font=image_font, fill=color)

    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

    return opencv_image



In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/summer-branch-410822-fc2d8cfced84.json'

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [24]:
path = '/content/img4.jpg'
with io.open(path, 'rb') as image_file:
    content = image_file.read()

In [25]:
image = vision.Image(content=content)

response = client.text_detection(image=image)
texts = response.text_annotations

In [26]:
# 텍스트와 바운딩 박스 정보를 담을 리스트
text_boxes = []

# 텍스트 바운딩 박스 좌표 출력
for i, text in enumerate(texts):
    if (i == 0):
      continue;

    vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
    # 중점 계산
    center_x = sum([v[0] for v in vertices]) / 4
    center_y = sum([v[1] for v in vertices]) / 4

    text_info = {
        "text": text.description,
        "bbox": vertices,
        "center": (center_x, center_y)
    }
    text_boxes.append(text_info)
    print(f'Text {i + 1}: {text_info["text"]}')
    print(f'Bounding Box Vertices===: {text_info["bbox"]}')
    print(f'Center===: {text_info["center"]}')
    print('---')

Text 2: Employee
Bounding Box Vertices===: [(37, 106), (170, 108), (169, 138), (36, 136)]
Center===: (103.0, 122.0)
---
Text 3: Name
Bounding Box Vertices===: [(177, 109), (256, 110), (255, 139), (176, 138)]
Center===: (216.0, 124.0)
---
Text 4: :
Bounding Box Vertices===: [(256, 111), (264, 111), (263, 140), (255, 140)]
Center===: (259.5, 125.5)
---
Text 5: Jerry
Bounding Box Vertices===: [(291, 111), (385, 113), (384, 143), (290, 141)]
Center===: (337.5, 127.0)
---
Text 6: Manager
Bounding Box Vertices===: [(41, 162), (156, 163), (156, 188), (41, 187)]
Center===: (98.5, 175.0)
---
Text 7: Name
Bounding Box Vertices===: [(166, 163), (242, 164), (242, 189), (166, 188)]
Center===: (204.0, 176.0)
---
Text 8: :
Bounding Box Vertices===: [(243, 164), (250, 164), (250, 188), (243, 188)]
Center===: (246.5, 176.0)
---
Text 9: Melinda
Bounding Box Vertices===: [(283, 169), (402, 171), (402, 191), (283, 189)]
Center===: (342.5, 180.0)
---
Text 10: Date
Bounding Box Vertices===: [(82, 287), (139

In [28]:
# 중점의 y 좌표를 기준으로 정렬
sorted_boxes_y = sorted(text_boxes, key=lambda x: x["center"][1])

# 바운딩 박스를 그룹화할 리스트
grouped_boxes = []

# 그룹화할 첫 번째 박스 추가
current_group = [sorted_boxes_y[0]]

# 그룹 내에서의 tolerance factor
group_tolerance_factor = 20

# 바운딩 박스를 그룹화
for i in range(1, len(sorted_boxes_y)):
    prev_center_y = current_group[-1]["center"][1]
    current_center_y = sorted_boxes_y[i]["center"][1]

    if abs(current_center_y - prev_center_y) < group_tolerance_factor:
        current_group.append(sorted_boxes_y[i])
    else:
        # 그룹 내에서 x 좌표를 기준으로 정렬
        sorted_group = sorted(current_group, key=lambda x: x["center"][0])
        grouped_boxes.extend(sorted_group)
        current_group = [sorted_boxes_y[i]]

# 마지막 그룹에 대한 정렬
sorted_group = sorted(current_group, key=lambda x: x["center"][0])
grouped_boxes.extend(sorted_group)

# 정렬된 바운딩 박스의 텍스트 출력
for box in grouped_boxes:
    print(box["text"])
    print(box["center"])


Employee
(103.0, 122.0)
Name
(216.0, 124.0)
:
(259.5, 125.5)
Jerry
(337.5, 127.0)
Manager
(98.5, 175.0)
Name
(204.0, 176.0)
:
(246.5, 176.0)
Melinda
(342.5, 180.0)
Week
(846.5, 179.0)
Starting
(943.0, 179.0)
:
(1001.5, 179.0)
6/23/2022
(1080.0, 179.0)
Date
(110.25, 296.0)
Day
(244.5, 301.0)
Time
(375.0, 299.5)
In
(425.0, 300.0)
Time
(539.0, 300.0)
Out
(598.0, 301.0)
Time
(734.5, 302.0)
In
(783.5, 301.5)
Time
(898.0, 300.0)
Out
(957.0, 301.0)
Total
(1078.5, 299.5)
Hours
(1152.0, 299.5)
6/23/2022
(112.5, 355.0)
Thursday
(246.5, 357.0)
08:30
(394.0, 358.5)
12:00
(564.5, 358.0)
13:20
(751.0, 361.5)
17:00
(926.5, 358.0)
0:00
(1117.5, 356.0)
6/24/2022
(116.0, 408.0)
Friday
(249.5, 410.0)
08:45
(394.0, 411.5)
11:58
(570.5, 411.5)
13:32
(755.0, 414.0)
17:50
(930.0, 410.0)
0:00
(1117.0, 408.0)
6/25/2022
(118.0, 458.0)
Saturday
(251.5, 459.0)
09:00
(399.0, 462.0)
12:30
(566.0, 463.0)
13:30
(753.0, 463.0)
17:55
(929.5, 461.0)
0:00
(1118.5, 456.5)
6/26/2022
(120.5, 506.5)
Sunday
(255.0, 508.5)
08:

In [29]:
# 중점의 y 좌표를 기준으로 정렬
sorted_boxes_y = sorted(text_boxes, key=lambda x: x["center"][1])

# 바운딩 박스를 그룹화할 리스트
grouped_boxes = []

# 그룹 내에서의 tolerance factor
group_tolerance_factor_y = 40
group_tolerance_factor_x = 300

# 바운딩 박스를 그룹화
current_group = [sorted_boxes_y[0]]

for i in range(1, len(sorted_boxes_y)):
    prev_center_y = current_group[-1]["center"][1]
    current_center_y = sorted_boxes_y[i]["center"][1]

    if abs(current_center_y - prev_center_y) < group_tolerance_factor_y:
        current_group.append(sorted_boxes_y[i])
    else:
        # 그룹 내에서 x 좌표를 기준으로 정렬
        sorted_group_x = sorted(current_group, key=lambda x: x["center"][0])

        # 형성된 그룹에서 새로운 그룹 형성
        new_group = [sorted_group_x[0]]

        for j in range(1, len(sorted_group_x)):
            prev_center_x = new_group[-1]["center"][0]
            current_center_x = sorted_group_x[j]["center"][0]

            if abs(current_center_x - prev_center_x) < group_tolerance_factor_x:
                new_group.append(sorted_group_x[j])
            else:
                # 같은 줄에 있는 바운딩 박스를 한 줄로 합치기
                line_text = ' '.join(box["text"] for box in new_group)
                grouped_boxes.append(line_text)

                new_group = [sorted_group_x[j]]

        # 마지막 그룹에 대한 정렬
        sorted_group_x = sorted(new_group, key=lambda x: x["center"][0])
        line_text = ' '.join(box["text"] for box in sorted_group_x)
        grouped_boxes.append(line_text)

        current_group = [sorted_boxes_y[i]]

# 마지막 그룹에 대한 정렬
sorted_group_x = sorted(current_group, key=lambda x: x["center"][0])
line_text = ' '.join(box["text"] for box in sorted_group_x)
grouped_boxes.append(line_text)

# 정렬된 바운딩 박스의 텍스트 출력
for line_text in grouped_boxes:
    print(line_text)


Employee Name : Jerry
Manager Name : Melinda
Week Starting : 6/23/2022
Date Day Time In Time Out Time In Time Out Total Hours
6/23/2022 Thursday 08:30 12:00 13:20 17:00 0:00
6/24/2022 Friday 08:45 11:58 13:32 17:50 0:00
6/25/2022 Saturday 09:00 12:30 13:30 17:55 0:00
6/26/2022 Sunday 08:57 11:56 13:40 18:00 0:00
6/27/2022 Monday 09:01 12:17 13:10 17:30 0:00
6/28/2022 Tuesday 08:27 12:08 13:30 17:51 0:00
6/29/2022 Wednesday 09:03 11:59 12:59 18:10 0:00
Total Hours 0:00
Employee Signature : Jerry
Rate Per Hour $ 15.00
Manager Signature : My
Total Pay $ 0.00
Brought to you by TimeDoctor.com
